In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
from holoviews import opts

from cross import cross_energy_matrix
from curvature import curvature_energy_matrix, segment_adjacent_pairs
from reconstruct import annealing_curve, update_layer_grad, energy, energy_gradient, should_stop
from plot import make_tracks_3d
from segment import gen_segments_all


In [ ]:
import pickle

with open('workdir/526420.pkl', 'rb') as fh:
    result = pickle.load(fh)

best_run = result.get_runs_by_id(result.get_incumbent_id())[-1]
best_run

In [ ]:
config = result.get_id2config_mapping()[best_run.config_id]['config']
config

In [ ]:
#config['alpha']= 30
# config['anneal_steps']= 87
# config['bias']= -5.660335705916375
# config['cosine_min_allowed']= -0.8360317823554324
# config['cosine_min_rewarded']=   0.934519555232472
# config['osine_power'] = 19.3757535534433   
# config['distance_power'] = 0.10073330110616618   
# config['dropout']=0    
# config['gamma']=4.820250024960375
# config['learning_rate']=0.4571332689260482
# config['max_hits']=800
# config['tarting_act']=0.8709695770981818
# config['threshold']=THRESHOLD=0.5
# config['tmax']=65.54559678237014
# config['tmin']=1.0
# config['total_steps']=100
# config['config_info']='model_based_pick'
# config

In [ ]:
from datasets import inputBMaN
simdata=inputBMaN() 
simdata.tail(3)


In [ ]:
from datasets import transformBMaN
hits = transformBMaN(simdata)
hits = hits[hits.event_id==6]
hits = hits[hits.track != -1]
hits = hits.reset_index().drop(columns='index')
hits

In [ ]:
def mark_track_segments(hits):
    track_segments = []
    for track, g in hits.groupby('track'):
        if track >= 0:
            for i in range(min(g.layer), max(g.layer)):
                for a in g[g.layer == i].index:
                    for b in g[g.layer == i + 1].index:
                        track_segments.append((a, b))
    return track_segments
track_segments = mark_track_segments(hits)
track_segments

In [ ]:
def build_tracks(hits):
    tracks = []
    for track, g in hits.groupby('track'):
        if track >= 0:
            tracks.append(list(g.index))
    return tracks
build_tracks(hits)

In [ ]:
def build_segmented_tracks(hits):
    tracks = []
    for track, g in hits.groupby('track'):
        if track >= 0:
            segments = []
            for i in range(min(g.layer), max(g.layer)):
                for a in g[g.layer == i].index:
                    for b in g[g.layer == i + 1].index:
                        segments.append((a, b))
            tracks.append((track, segments))
    return dict(tracks)
all_tracks = build_segmented_tracks(hits)
all_tracks

In [ ]:
# event[event.track_id==373]

In [ ]:
np.all(np.array([1,2,3]) == (1,2,3))

In [ ]:
track_segments = mark_track_segments(hits)
len(track_segments)

In [ ]:
pos = hits[['x', 'y', 'z']].values

In [ ]:
seg = gen_segments_all(hits)
len(track_segments), len(seg)

In [ ]:
perfect_act = np.zeros(len(seg))
track_segment_set = set(tuple(s) for s in track_segments)
is_in_track = np.array([tuple(s) in track_segment_set for s in seg])
perfect_act[is_in_track] = 1

In [ ]:
pairs = segment_adjacent_pairs(seg)

crossing_matrix = cross_energy_matrix(seg, pos, config['cosine_min_allowed'])

curvature_matrix = curvature_energy_matrix(pos, seg, pairs,
                                           config['cosine_power'], config['cosine_min_rewarded'],
                                           config['distance_power'])

e_matrix = config['alpha'] / 2 * crossing_matrix - config['gamma'] / 2 * curvature_matrix

In [ ]:
temp_curve = annealing_curve(config['tmin'], config['tmax'],
                             config['anneal_steps'], config['total_steps'] - config['anneal_steps'])

tcdf = pd.DataFrame({'temp': temp_curve})
tcdf.index.name = 'step'
tcdf.plot()

In [ ]:
act = np.full(len(seg), config['starting_act'])
acts = [act.copy()]
for i, t in enumerate(temp_curve):
    grad = energy_gradient(e_matrix, act)
    update_layer_grad(act, grad, t, config['dropout'], config['learning_rate'], config['bias'])
    acts.append(act.copy())

In [ ]:
from typing import List, Tuple
def enumerate_segmented_track(track: List[Tuple[int, int]], seg: np.array) -> List[int]:
    nom_seg=[]
    for step in track:  
        z = np.zeros_like(seg)
        condition = ((seg[:, 0] == step[0]) & (seg[:, 1] == step[1]))
        tek_nom_seg=np.arange(len(seg))[condition][0]
        nom_seg.append(tek_nom_seg)
    return nom_seg

# test 1
test_tracks = enumerate_segmented_track(
    [(21, 44), (44, 79), (79, 123), (123, 164), (164, 190), (190, 209), (209, 228), (228, 242)],
    np.array([(21, 44), (44, 79), (79, 123), (123, 164), (164, 190), (190, 209), (209, 228), (228, 242)])
)
print(test_tracks)
assert test_tracks == [0, 1, 2, 3, 4, 5, 6, 7]

# test 2
test_tracks = enumerate_segmented_track([(0, 1), (1, 3)], np.array([(0, 1), (0, 2), (1, 3)]))
print(test_tracks)
assert test_tracks == [0, 2]

In [ ]:
def found_tracks(seg: np.ndarray, act: np.ndarray, all_tracks: List[List[Tuple[int, int]]]) -> int:
    
    kol_act_track=0
    all_kol_track=0
    end_track=False
    i=0
    lseg=[]
    for ti, t in (enumerate(all_tracks)):
        kol_act_seg=0

        all_kol_track+=1
        Track_seg=enumerate_segmented_track(t, seg)

        for s in Track_seg:
            if (act[s] >= config['threshold']):
                kol_act_seg+=1
            
        if kol_act_seg==len(Track_seg):    
            kol_act_track+=1
                
#         print("All active segments    "+str(kol_act_seg))
#         print("All active tracks      "+str(kol_act_track))
#         print("All tracks for exp BM@N "+str(all_kol_track))
    return kol_act_track



# %prun found_tracks(seg, act, all_tracks.values())
found_tracks(seg, act, all_tracks.values())

In [ ]:
def found_crosses(seg: np.ndarray, act: np.ndarray) -> int:
    kol_crosses=0
    seg_tek=seg[act >= config['threshold']]
    for si, s in enumerate(seg):
        if (act[si] >= config['threshold']):
            condition1 = (seg_tek[:,0]  == s[0]) & (seg_tek[:,1] != s[1] )
            kol_crosses += np.sum(condition1)
            condition2 = (seg_tek[:,0]!= s[0]) & (seg_tek[:,1] == s[1])
            kol_crosses += np.sum(condition2)
    return(kol_crosses//2)

_seg = np.array([(0, 2), (0, 3), (1, 2), (1, 3)])
assert found_crosses(_seg, np.array((0, 0, 0, 0))) == 0
assert found_crosses(_seg, np.array((1, 0, 0, 1))) == 0
assert found_crosses(_seg, np.array((0, 1, 1, 0))) == 0
assert found_crosses(_seg, np.array((1, 0, 1, 0))) == 1
assert found_crosses(_seg, np.array((1, 1, 0, 0))) == 1
assert found_crosses(_seg, np.array((1, 1, 1, 1))) == 4
                
# %prun found_crosses(seg, act)
found_crosses(seg, act)

In [ ]:
energy_history = []
for act in acts:
    ef = config['alpha'] / 2 * energy(crossing_matrix, act)
    ec = config['gamma'] / 2 * energy(curvature_matrix, act) # inverted to positive
    energy_history.append([ec, ef, ec+ef])
energy_history = pd.DataFrame(energy_history, columns=['E_curve', 'E_fork', 'E'])
energy_history.index.name = 'step'
energy_history.plot(logy=True)
pass

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
small_history = pd.DataFrame([
    (
        precision_score(perfect_act, act>config['threshold'], zero_division=0),
        recall_score(perfect_act, act>config['threshold'], zero_division=0),
        f1_score(perfect_act, act>config['threshold'], zero_division=0),
        roc_auc_score(perfect_act, act),
        average_precision_score(perfect_act, act),
        act.mean(),
        ((act - perfect_act) ** 2).mean()
    ) for act in acts],
    columns=['precision', 'recall', 'f1', 'roc_auc', 'ap', 'mean_act', 'dist_perfect'])
small_history.index.name = 'step'
small_history.plot()

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score
average_precision = average_precision_score(perfect_act, act)
precision, recall, _ = precision_recall_curve(perfect_act, act)
PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision).plot()

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay
fpr, tpr, _ = roc_curve(perfect_act, act)
RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
from sklearn.metrics import det_curve, DetCurveDisplay
fpr, fnr, _ = det_curve(perfect_act, act)
DetCurveDisplay(fpr=fpr, fnr=fnr).plot()

In [ ]:
n_steps = min(16, len(acts))
steps = np.linspace(0, len(acts) - 1, n_steps, dtype=int)

tracks_3d = []
tracks_projection = []
tracks_by_track = []
for i in steps:
    tp, fp, tn, fn = make_tracks_3d(pos, seg, acts[i], perfect_act, config['threshold'])
    vdims = ['act', 'perfect_act', 'positive', 'true']
    xyz = hv.Overlay([
        hv.Scatter3D(hits[hits.track == -1], kdims=['x', 'y', 'z'], label='noise', group='hits'),
        hv.Scatter3D(hits[hits.track != -1], kdims=['x', 'y', 'z'], label='hits', group='hits'),
        hv.Path3D(tp, vdims=vdims, label='tp', group='tracks'),
        hv.Path3D(fp, vdims=vdims, label='fp', group='tracks'),
        hv.Path3D(fn, vdims=vdims, label='fn', group='tracks')
    ])
    tracks_3d.append(xyz)

    projection = hv.Points(hits[hits.track != -1], kdims=['x', 'y'], label='hits', group='hits') * \
                 hv.Points(hits[hits.track == -1], kdims=['x', 'y'], label='noise', group='hits') * \
                 hv.Path(tp, kdims=['x', 'y'], vdims=vdims, label='tp', group='tracks') * \
                 hv.Path(fp, kdims=['x', 'y'], vdims=vdims, label='fp', group='tracks') * \
                 hv.Path(fn, kdims=['x', 'y'], vdims=vdims, label='fn', group='tracks')
    tracks_projection.append(projection)

    nodes = hv.Nodes(hits, kdims=['track', 'layer', 'index'])
    no_tn = np.logical_or(acts[i] > config['threshold'], perfect_act > config['threshold'])
    graph = hv.Graph(((*seg[no_tn].transpose(), acts[i][no_tn], perfect_act[no_tn]), nodes),
                     vdims=['act', 'perfect_act'])
    tracks_by_track.append(hv.Overlay([graph]))


In [ ]:
hv.extension('matplotlib')
tracks_3d[-1].opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='red'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', fig_size=400),
)

In [ ]:
hv.extension('bokeh')
track_history = \
    hv.HoloMap(
        {s: hv.Histogram(np.histogram(acts[s][perfect_act > config['threshold']], bins=32, range=(0, 1))) for s in
         np.linspace(0, len(acts) - 1, min(64, len(acts)), dtype=int)},
        kdims='step', group="Activation", label="On track") + \
    hv.HoloMap(
        {s: hv.Histogram(np.histogram(acts[s][perfect_act < config['threshold']], bins=32, range=(0, 1))) for s in
         np.linspace(0, len(acts) - 1, min(64, len(acts)), dtype=int)},
        kdims='step', group="Activation", label="Off track")

# track_history.opts(
#     opts.Histogram(width=400, height=400)
# )

In [ ]:
hv.extension('plotly')
tracks_3d[-1].opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='red'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=800, height=800)
)

In [ ]:
hv.extension('matplotlib')
track_history = hv.NdLayout(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='red'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right'),
    opts.NdLayout(fig_size=200)
).cols(4)

In [ ]:
hv.extension('plotly')
track_history = hv.HoloMap(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='red'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=700, height=700)
)
# hv.save(track_history, 'track_history_3d.html', fmt='widgets')

In [ ]:
# hv.extension('matplotlib')
# track_history = hv.HoloMap(
#     {s: tracks_projection[i] for i, s in enumerate(steps)},
#     kdims='step'
# )
# track_history.opts(
#     opts.Points(color='black'),
#     opts.Points('Hits.Noise', color='black'),
#     opts.Path(color='black', show_legend=True),
#     opts.Path('Tracks.fp', color='red'),
#     opts.Path('Tracks.fn', color='cyan'),
#     opts.Overlay(legend_position='right', fig_size=256),
# )

In [ ]:
hv.extension('bokeh')
track_history = hv.HoloMap(
    {s: tracks_projection[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Points(color='black', size=4),
    opts.Points('Hits.Noise', color='red'),
    opts.Path(color='black', show_legend=True),
    opts.Path('Tracks.fp', color='red'),
    opts.Path('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', width=700, height=700)
)


In [ ]:
hits_to_seg = {tuple(pair): i for i, pair in enumerate(seg)}
    
for k,v in hits_to_seg.items():
    print (k,v)

In [ ]:
import pandas as pd
from typing import Dict, Tuple

def is_track_found(hits_group: pd.DataFrame, hits_to_seg: Dict[Tuple[int, int], int], act: np.ndarray):
    found=[]
    hits_group = hits_group.sort_values('layer')
    for i in range(len(hits_group) - 1):
        h1 = hits_group.index[i]
        j = i + 1
        while (hits_group.layer.iloc[i] == hits_group.layer.iloc[j]):
            j += 1
            if j>len(hits_group)-1:
                break
        if j>len(hits_group)-1:
            break
        if hits_group.layer.iloc[i] + 1 != hits_group.layer.iloc[j]:
            continue
        h2 = hits_group.index[j]
        s = hits_to_seg[h1, h2]
        found.append(act[s]>config['threshold'])
    return all(found)


is_track_found(hits[hits.track==2], hits_to_seg, acts[-1])

found = hits[hits.track != -1].groupby('track').apply(is_track_found,hits_to_seg,act)
found.sum(), len(found), found.sum()/len(found)